In Python we can use variables, which can store simple values, as string, numbers and booleans (yes/no values)

In [ ]:
name = "david"
age = 50
married = True

We can use and display those values

In [ ]:
print(name)
print(name + " is " + str(age) + " years old")
if married:
    print("And he is married")
else:
    print("And he is single")

In Jupyter Notebooks (not in normal Python programs) if a cell ends in an expression, the expression gets displayed

In [ ]:
name

In [ ]:
name.upper()

You can also have lists of values

In [ ]:
hobbies = ["Books", "Movies", "Computers"]
hobbies

And you can have dictionaries, an object with several keys and the corresponding value for that key
Similar to a record in a database

In [ ]:
david = {
    "first_name": "david",
    "last_ name": "sol",
    "age": 50,
    "married": True
}
david

You can even have "complex" data

In [ ]:
david["hobbies"] = hobbies
david

Dictionaries can be exported or imported from JSON, a format to interchange information (like comma-separated value files or XML)

In [ ]:
import json
json.dumps(david)

Woth `import` you can add modules with a lot of code into Python

In [ ]:
import this

The `boto3` module let you use Amazon Web Services (AWS) in a very simple way

In [ ]:
import boto3

We need and Access Key Id and Access Key Secret to authenticate, use yours or ask for it to your instructor

## WARNING: You should NEVER store credentials in a repository. Don't save the notebook with the credentials

In [ ]:
access_id = ""
secret_key = ""

We open a session

In [ ]:
aws_session = boto3.Session(
    aws_access_key_id = access_id,
    aws_secret_access_key = secret_key,
    region_name = "us-west-2"
)

We open a client to the Rekognition service, Artificial Intelligence vision

In [ ]:
rekognition_client = aws_session.client('rekognition')

We set up the image name

In [ ]:
image_name = "sample.jpg"

We import the module for image processing

In [ ]:
from PIL import Image

We open the image, and display it

In [ ]:
image_display = Image.open(image_name)

In [ ]:
image_display

Now we will identify objects in the image (AWS calls it "tagging"). We load the image and send it out to AWS for analysis

In [ ]:
with open(image_name, 'rb') as image:
        response = rekognition_client.detect_labels(Image={'Bytes': image.read()})

The response is a dictionary, with a lot of values

In [ ]:
print(response)

To better understand it we "Pretty print" it

In [ ]:
from pprint import pprint

In [ ]:
pprint(response)

Let's concentrate in just one part.
The identified labels are in the Label field, with is a list of dictionaries

In [ ]:
for label in response["Labels"]:
    print(label["Name"])

Let's dig into the first label

In [ ]:
pprint(response["Labels"][0])

Now the fun begins

In [ ]:
import io
from PIL import Image, ImageDraw, ImageFont

In [ ]:
font = ImageFont.truetype('Consolas', size=20)
draw = ImageDraw.Draw(image_display)
# Get all labels
w, h = image_display.size

In [ ]:
name = response["Labels"][0]["Name"]
print(name)
instance = response["Labels"][0]["Instances"][0]
pprint(instance)

In [ ]:
bbox = instance['BoundingBox']
x0 = int(bbox['Left'] * w) 
y0 = int(bbox['Top'] * h)
x1 = x0 + int(bbox['Width'] * w)
y1 = y0 + int(bbox['Height'] * h)
draw.rectangle([x0, y0, x1, y1], outline=(255, 0, 0), width=2)
draw.text((x0, y0), name, font=font, fill=(0, 255, 255))
image_display

In [ ]:
for label in response['Labels']:
    name = label['Name']
    # Draw all instancex box, if any
    for instance in label['Instances']:
        bbox = instance['BoundingBox']
        x0 = int(bbox['Left'] * w) 
        y0 = int(bbox['Top'] * h)
        x1 = x0 + int(bbox['Width'] * w)
        y1 = y0 + int(bbox['Height'] * h)
        draw.rectangle([x0, y0, x1, y1], outline=(255, 0, 0), width=2)
        draw.text((x0, y0), name, font=font, fill=(0, 255, 255))

In [ ]:
image_display

In [ ]:
image_display.save('labels.jpg')